## 📚 Prerequisites

Before executing this notebook, make sure you have properly set up your Azure Services, created your Conda environment, and configured your environment variables as per the instructions provided in the [README.md](README.md) file.

## 📋 Table of Contents

This notebook guides you through the following sections:

> **💡 Note:** Please refer to the notebook `01-creation-indexes.ipynb` for detailed information and steps on how to create Azure AI Search Indexes.

1. [**Indexing Vectorized Content from Documents**](#index-documents)
    - Chunk, vectorize, and index local PDF files and website addresses.
    - Download, chunk, vectorize, and index all `.docx` files from a SharePoint site.
    - Download PDF files stored in Blob Storage, apply complex OCR processing through GPT-4 Vision, chunk and vectorize the content, and finally index the processed data in Azure AI Search.
    
2. [**Indexing Vectorized Content from Domanin knowledge document containing complex Documents Images or more**](#index-images)
    - Leverage complex OCR, image recognition, and summarization capabilities using GPT-4 Vision. Chunk, vectorize, and index extracted metadata from images stored in Blob Storage.

3. [**Indexing Vectorized Content from Audio**](#index-audio)
    - Process WAV audio data using Azure AI Speech Translator capabilities, chunk, vectorize, and index audio files stored in Blob Storage and indexed in Azure AI Search.

In [1]:
import os

# Define the target directory
target_directory = r"C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing"  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing


# Create Azure AI Search Indexes 

Please refer to the notebook [01-creation-indexes.ipynb](01-creation-indexes.ipynb) for detailed information and steps on how to create Azure AI Search Indexes. 

# Indexing Vectorized Content from Multiple Sources and Various Formats

In [2]:
# Import the AzureAIndexer class from the ai_search_indexing module
from src.indexers.ai_search_indexing import AzureAIndexer

DEPLOYMENT_NAME = "foundational-ada"
INDEX_NAME = "test-index-002"

# Create an instance of the AzureAIndexer class
azure_search_indexer_client = AzureAIndexer(
    index_name=INDEX_NAME, embedding_azure_deployment_name=DEPLOYMENT_NAME
)

2024-01-13 20:49:12,528 - micro - MainProcess - INFO     Loading OpenAIEmbeddings object with model, deployment foundational-ada, and chunk size 1000 (ai_search_indexing.py:load_embedding_model:151)
c:\Users\pablosal\AppData\Local\anaconda3\envs\vector-indexing-azureaisearch\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAIEmbeddings instead.
  warn_deprecated(
c:\Users\pablosal\AppData\Local\anaconda3\envs\vector-indexing-azureaisearch\lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(
2024-01-13 20:49:13,766 - micro - MainProcess - INFO     AzureOpenAIEmbeddings object has been created successfully. You can now access the embed

## Indexing Pdfs and Docx from Blob Storage 

In the process of indexing and processing documents, the first crucial step is loading the files and splitting them into manageable chunks. This is where the `load_files_and_split_into_chunks` function comes into play. This function is designed to streamline these initial steps, preparing your documents for further processing and conversion.

Here are its key features:

- **Multi-Format Support**: The function can seamlessly process documents in different formats (PDFs, Word documents, etc.) from an array of sources (blob storage, URLs, local paths). You can pass a list of file paths, each possibly in a different format.

- **Automated File Loading**: The function efficiently loads files into memory, eliminating the need for manual file handling. It manages the reading and processing of each file.

- **Advanced Text Splitting**: After loading, the function splits the text into manageable chunks, which is crucial for processing large documents. You can customize the chunk size and overlap according to your needs.

- **Versatile Splitting Options**: You can choose from various splitters - RecursiveCharacterTextSplitter, TokenTextSplitter, SpacyTextSplitter, or CharacterTextSplitter - to fit your specific text processing requirements.

- **Encoding Capabilities**: The function can optionally use an encoder during splitting. This feature is particularly useful for certain text analysis tasks. You can specify the model used for encoding (default is "gpt-4").

- **Verbose Logging**: You can enable detailed logging for in-depth progress tracking and easier debugging.

- **High Customizability**: The function's behavior can be tailored to your needs with additional keyword arguments. This includes options like retaining separators in chunks, using separators as regex patterns, and more.

In [3]:
# Define file paths and URLs
local_pdf_path = "utils/data/autogen.pdf"
remote_pdf_url = "https://arxiv.org/pdf/2308.08155.pdf"
blob_pdf_url = (
    "https://testeastusdev001.blob.core.windows.net/testretrieval/autogen.pdf"
)
local_word_path = "utils/data/test.docx"
remote_word_url = (
    "https://testeastusdev001.blob.core.windows.net/testretrieval/test.docx"
)

# Combine all paths and URLs into a list. This is optional if you want to process multiple files at once.
# It will also work by passing a string for simple file processing.
file_sources = [
    local_pdf_path,
    remote_pdf_url,
    blob_pdf_url,
    local_word_path,
    remote_word_url,
]

# Define parameters for the load_files_and_split_into_chunks function
splitter_params = {
    "splitter_type": "recursive",
    "use_encoder": False,
    "chunk_size": 512,
    "chunk_overlap": 128,
    "verbose": False,
    "keep_separator": True,
    "is_separator_regex": False,
    "model_name": "gpt-4",
}

# Load files and split them into chunks
document_chunks_to_index = azure_search_indexer_client.load_files_and_split_into_chunks(
    file_paths=file_sources, **splitter_params
)

2024-01-13 20:45:01,085 - micro - MainProcess - INFO     Reading .pdf file from local path C:\Users\pablosal\Desktop\gbbai-azure-ai-search-indexing\utils\data\autogen.pdf. (loading.py:load_file:107)
2024-01-13 20:45:01,087 - micro - MainProcess - INFO     Loading file with Loader PyPDFLoader (loading.py:load_file:117)
2024-01-13 20:45:03,415 - micro - MainProcess - INFO     Reading .pdf file from https://arxiv.org/pdf/2308.08155.pdf. (loading.py:load_file:113)
2024-01-13 20:45:03,416 - micro - MainProcess - INFO     Loading file with Loader PyPDFLoader (loading.py:load_file:117)
2024-01-13 20:45:06,993 - micro - MainProcess - INFO     Reading .pdf file from temporary location C:\Users\pablosal\AppData\Local\Temp\tmpv0rrugjr originally sourced from https://testeastusdev001.blob.core.windows.net/testretrieval/autogen.pdf. (loading.py:load_file:105)
2024-01-13 20:45:06,994 - micro - MainProcess - INFO     Loading file with Loader PyPDFLoader (loading.py:load_file:117)
2024-01-13 20:45:08,

In [5]:
# Index the document chunks using the Azure Search Indexer client
azure_search_indexer_client.index_text_embeddings(document_chunks_to_index)

2024-01-13 14:39:02,167 - micro - MainProcess - INFO     Embedding and indexing initiated for 1408 text chunks. (ai_search_indexing.py:index_text_embeddings:342)
2024-01-13 14:41:20,904 - micro - MainProcess - INFO     Embedding and indexing completed for 1408 text chunks. (ai_search_indexing.py:index_text_embeddings:344)


## Indexing Pdfs and Docs from Sharepoint


In [3]:
file_names = ["testdocx.docx"]

In [4]:
azure_search_indexer_client.load_files_and_split_into_chunks_from_sharepoint(
    site_domain=os.environ["SITE_DOMAIN"],
    site_name=os.environ["SITE_NAME"],
    file_names=file_names,
)

2024-01-13 20:49:34,134 - micro - MainProcess - INFO     Loading environment variables from .env file. (loading.py:load_file_from_sharepoint:189)
2024-01-13 20:49:34,140 - micro - MainProcess - INFO     Successfully loaded environment variables: TENANT_ID, CLIENT_ID, CLIENT_SECRET (sharepoint_data_extractor.py:load_environment_variables_from_env_file:86)
2024-01-13 20:49:34,141 - micro - MainProcess - INFO     Authenticating with Microsoft Graph. (loading.py:load_file_from_sharepoint:192)
2024-01-13 20:49:34,643 - micro - MainProcess - INFO     New access token retrieved. (sharepoint_data_extractor.py:msgraph_auth:119)
2024-01-13 20:49:34,644 - micro - MainProcess - INFO     Getting site and drive IDs. (loading.py:load_file_from_sharepoint:199)
2024-01-13 20:49:34,645 - micro - MainProcess - INFO     Getting the Site ID... (sharepoint_data_extractor.py:get_site_id:189)
2024-01-13 20:49:35,381 - micro - MainProcess - INFO     Site ID retrieved: mngenvmcap747548.sharepoint.com,877fe60f-a

[]